In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import json
train_annotations_path = "/kaggle/input/vizwiz/Annotations/Annotations/train.json" 
val_annotations_path = "/kaggle/input/vizwiz/Annotations/Annotations/val.json"  
image_dir = "/kaggle/input/vizwiz/train/train"

In [23]:
with open(train_annotations_path, 'r') as f:
    train = json.load(f)
train_df = pd.DataFrame(train)

with open(val_annotations_path, 'r') as f:
    val = json.load(f)

val_df = pd.DataFrame(val)

In [25]:
train_df['image'] = '/kaggle/input/vizwiz/train/train/' + train_df['image']

In [26]:
val_df['image'] = '/kaggle/input/vizwiz/val/val/' + val_df['image']

In [27]:
import pandas as pd
import cv2

def detect_blurry_images_in_dataset(dataset, image_column, prefix, blur_threshold=100.0, min_width=256, min_height=256):
    """
    Flags blurry and low-quality images in a dataset.

    Args:
        dataset (pd.DataFrame): DataFrame containing the dataset.
        image_column (str): Column name with image paths.
        blur_threshold (float): Minimum Laplacian variance to consider an image sharp.
        min_width (int): Minimum width of images.
        min_height (int): Minimum height of images.

    Returns:
        pd.DataFrame: Updated DataFrame with quality flags.
    """
    blurry_flags = []
    resolution_flags = []

    for image_path in dataset[image_column]:
        try:
            image = cv2.imread(prefix + image_path)
            if image is None:
                print(f"Could not read image {image_path}. Marking as invalid.")
                blurry_flags.append(True)
                resolution_flags.append(True)
                continue

            height, width = image.shape[:2]
            if width < min_width or height < min_height:
                resolution_flags.append(True)
            else:
                resolution_flags.append(False)

            # Compute Laplacian variance for blur detection
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            blurry_flags.append(laplacian_var < blur_threshold)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            blurry_flags.append(True)
            resolution_flags.append(True)

    dataset['is_blurry'] = blurry_flags
    dataset['is_low_quality'] = resolution_flags

    return dataset

In [28]:
t_df = detect_blurry_images_in_dataset(train_df, image_column='image',prefix ='', blur_threshold=100.0, min_width=128, min_height=128)
v_df = detect_blurry_images_in_dataset(val_df, image_column='image',prefix ='', blur_threshold=100.0, min_width=128, min_height=128)

In [29]:
t_df = t_df[t_df['is_blurry'] == False]
t_df = t_df[t_df['is_low_quality'] == False]

v_df = v_df[v_df['is_blurry'] == False]
v_df = v_df[v_df['is_low_quality'] == False]

In [30]:
import os
t_df['image'] = t_df['image'].apply(os.path.basename)
v_df['image'] = v_df['image'].apply(os.path.basename)

In [31]:
pd.DataFrame(t_df['image']).to_csv('filtered_data.csv')
pd.DataFrame(v_df['image']).to_csv('filtered_data_val.csv')